In [7]:
from openai import OpenAI
import json
import os

GPT_MODEL_3_5 = "gpt-3.5-turbo-1106"
GPT_MODEL_4 = "gpt-4-0125-preview"

OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_3_5

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return (response.choices[0].message.content)

no_condition  = False
no_non_condition_flow = False
filter_flow = []

default_flow = "Basic flow"


In [8]:
def read_file_into_string(file_path):
    try:
        with open(file_path, 'r') as file:
            file_content = file.read()
        return file_content
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
        return None
    except Exception as e:
        print(f"Error reading file: {e}")
        return None


In [9]:
def write_to_file(filename, s):
    try:
        with open(filename, 'w') as file:
            file.write(s)
        print("Content has been written to", filename)
    except Exception as e:
        print("An error occurred:", e)

In [10]:
file_path_list = [
"D:\GPT-testing\SpecificationData\Book(Github)\Display genre details.txt",
"D:\GPT-testing\SpecificationData\Book(Github)\Display the list of authors.txt",
"D:\GPT-testing\SpecificationData\Book(Github)\Edit the author.txt",
"D:\GPT-testing\SpecificationData\Book(Github)\Remove the author.txt",
"D:\GPT-testing\SpecificationData\Book(Github)\Add a new author.txt",
]


In [11]:
SYSTEM_PROMPT_extract_flow ="""
Given use case specification.
Extract all the contents of flows and identify all the interactive elements within the feature into json format.
For each interactive element:
Identify what type of that element (button,buttons, icon,scroll, text field,text area, tab, radio buttons, menu, combobox, sliders, switches, dialog, link, form,rating, filter).
Identify all the conditions mentioned in the use case of that element that would make the element valid and the conditions that would make the element invalid based on the description of the use case flow.
Do not arbitrarily create additional conditions that not mention in the use case flow.
Return the result in json format.
{
"Flow": {
  Flow name:[ contents of corresponding flow]
  },
"Element extract":{
  Name of interactive element: 
    {"condition": {"valid":conditions that make element valid, "invalid": conditions that make element invalid}, "type": element type}}
  }
}
An example output: 
{
  "Flow": {
    "Main flow:": [
      "Step 1: Learner fills in the username field by a valid username that has been registered",
      "Step 2: Learner fills in the password field by the correct password for the corresponding username",
      "Step 3: Learner press \"Login\" button",
      "Step 4: System redirects learner to Home page"
    ],
    "Alternative flow 1: Login by email": [
      "At step 1 of the basic flow: Learner fills in the username field by a valid email that has been registered",
      "Step 2: Learner fills in the password field by the correct password for the corresponding email",
      "Go back to step 3 in the basic flow and continue with the steps from step 3"
    ]
  }
  "Element extract":{
    {"Username": {"condition": {"valid": "must be over 8 characters and below 30 characters, must be entered", racter, empty"}, "type": "text field"}}
    {"Search button": {"condition": {"valid": "", "invalid": ""}, "type": "text field"}}
  }
}
"""

In [12]:
# Example usage:
# file_path = "D:\GPT-testing\SpecificationData\Adventura - Flow only\Adventura - Filter.txt"
for file_path in file_path_list:
    usecase = "Use case: "+ read_file_into_string(file_path)
    if usecase is not None:
        print(usecase)
    promptExtractCondtition = [
        { "role": "system", "content": SYSTEM_PROMPT_extract_flow},
        { "role": "user", "content": usecase}
    ]

    gpt_response = ask(promptExtractCondtition, client, model)
    print(file_path)
    print(gpt_response)
    print()
    print()

    usecase_flow = json.loads(gpt_response)
    flows_name = [key for key in usecase_flow]

Use case: Display genre details

Main scenario:
User clicks on the "Name" of the specific genre from the list on the genre list page;
Application displays the user-chosen genre details;

On genre details page the application displays the following columns:
id - unique genre number
Name
Description
D:\GPT-testing\SpecificationData\Book(Github)\Display genre details.txt
{
  "Flow": {
    "Main scenario: Display genre details": [
      "User clicks on the 'Name' of the specific genre from the list on the genre list page",
      "Application displays the user-chosen genre details"
    ]
  },
  "Element extract": {
    "Name": {
      "condition": {
        "valid": "User clicks on the 'Name' of the specific genre from the list on the genre list page",
        "invalid": ""
      },
      "type": "link"
    }
  }
}


Use case: Display the list of authors
Description
The page is designed to list all the authors.

Main scenario:
User selects item "Authors";
Application displays the list of th